In [1]:
!pip install pandas
!pip install numpy
!pip install gensim
!pip install gdown

In [1]:
#imports
import numpy as np
import pandas as pd
import gdown
import gzip
import shutil

In [2]:
#Downloads the pretrained word2vec model and extract it, you only need to run this once to obtain the file.
url = "https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM"
output = "GoogleNews-vectors-negative300.bin.gz"
gdown.download(url, output,quiet = False)
with gzip.open('GoogleNews-vectors-negative300.bin.gz', 'rb') as f_in:
    with open('GoogleNews-vectors-negative300.bin', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

Downloading...
From (uriginal): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=7553074d-827e-46bf-808f-e162ecc8ee16
To: D:\Old Projects\Synthetic-Dataset-Creation-for-NLP\GoogleNews-vectors-negative300.bin.gz
100%|█████████████████████████████████████████████████████████████████████████████| 1.65G/1.65G [06:12<00:00, 4.42MB/s]


In [3]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary = True) #Loads in pretrained word2vec model

In [4]:
colnames = ["input","label"]
ds = pd.read_csv('training_data.csv', usecols = colnames)
with open("stoplist.txt", "r") as F:
    stoplist = F.read()
    clean = []
for i in range(0, len(ds["input"])):
    clean.append([word for word in ds["input"][i].split() if word not in stoplist])

dataset = []
temp = []
label = []
for i in clean:
    for j in i:
        for k in range(0,10):
            word = model.most_similar(j)[k][0]
            temp = i.copy()
            temp[i.index(j)] = word
            dataset.append(temp)
            label.append(ds['label'][clean.index(i)])#Dataset expansion

In [5]:
sentence = ""
sentences = []
for i in dataset:
    for j in i:
        if i.index(j) == 0:
            sentence = sentence + j
        else:
            sentence = sentence + " " + j
    sentences.append(sentence)
    sentence = ""
    
df = pd.DataFrame({
    'input': sentences,
    'label': label
})
df.to_csv('synthetic_training_data.csv',index = False)#Inserting the expanded dataset.